## Ground Truth Generation Notebook.

This notebook generates a pre configured number of molecules and shows their alignments

In [1]:
Molecule = 1 #Pick molecule Number between 0 and 512

### Imports 

In [2]:
%load_ext autoreload
%autoreload 2

import numpy
from scipy.interpolate import interp1d
from deepom.bionano_utils import MoleculeSelector

from deepom.config import Config
from deepom.localizer import LocalizerModule
import monai

from deepom.aligner import Aligner
from deepom.my_utils import get_aligner, overlap_percentage, get_scaler_from_aligner
import numpy as np

from deepom.bionano_compare import BionanoCompare
from matplotlib.pyplot import eventplot, imshow, figure, xlim, title




### selector Creation

In [3]:
#Need To check about Caching
selector = MoleculeSelector()
selector.top_mol_num = 5
selector.select_molecules()


KeyboardInterrupt



### Refs

In [ ]:
compare = BionanoCompare()
compare.read_cmap()
compare.make_refs()
compare.refs

In [ ]:
selector.selected[Molecule].locs

### Localizer

In [ ]:
localizer_module = LocalizerModule()
localizer_module.checkpoint_search_dir = Config.CHECKPOINT_SEARCH_DIR
localizer_module.load_checkpoint = True
localizer_module.init_ensure_module()

image_input = selector.selected[Molecule].bionano_image.segment_image[0]
target_width = localizer_module.image_channels
source_width = image_input.shape[0] // 2 + 1
image_input = image_input[source_width - target_width // 2: source_width + target_width // 2 + 1]
inference_item = localizer_module.inference_item(image_input)

### Alignment

In [ ]:
ref_id = selector.selected[Molecule].xmap_item.ref_id

aligner_reg = get_aligner()
aligner_rev = get_aligner()

qry = inference_item.loc_pred * 335
offset = image_input.shape[-1]-inference_item.loc_pred[-1]
inverted_qry = np.sort((image_input.shape[-1] - inference_item.loc_pred)*335)#not sure, need to make sure its correct

aligner_reg.make_alignment(qry=qry, ref=compare.refs[ref_id])
aligner_rev.make_alignment(qry=inverted_qry, ref=compare.refs[ref_id])

### Selecting the best orientation based on overlap percentage

In [ ]:
regular_overlap = overlap_percentage(selector.selected[Molecule].xmap_item.ref_lims, aligner_reg.alignment_ref[[0, -1]])
print("regular overlap: "+str(regular_overlap))
print("regular score: "+str(aligner_reg.score))

reverse_overlap = overlap_percentage(selector.selected[Molecule].xmap_item.ref_lims, aligner_rev.alignment_ref[[0, -1]])
print("reverse ovelap: "+str(reverse_overlap))
print("reverse score: "+str(aligner_rev.score))

aligner = aligner_reg if regular_overlap > reverse_overlap else aligner_rev

selector.selected[Molecule].xmap_item.ref_lims, aligner.alignment_ref

In [ ]:
scaler = get_scaler_from_aligner(aligner)

z = aligner.alignment_ref
y = (z-z[0]) / (scaler*335)
y = y+aligner.alignment_qry[0]/335
x = aligner.alignment_qry/335
func = interp1d(x=y,y=x)

ref = compare.refs[ref_id]
start, end = aligner.alignment_ref[[0, -1]]
start_index = np.argmax(ref >= start)
end_index = np.argmin(ref <= end)
ref_crop = ref[start_index:end_index]

z_hat = (ref_crop-ref_crop[0]) / (scaler*335)
z_hat = z_hat+x[0]
y_hat = func(z_hat)

### Plot

In [ ]:
figure(figsize=(30, 3))
image_input = selector.selected[Molecule].bionano_image.segment_image[0]
target_width = localizer_module.image_channels
source_width = image_input.shape[0] // 2 + 1
image_input = image_input[source_width - target_width // 2: source_width + target_width // 2 + 1]
imshow(image_input, aspect="auto", cmap="gray")
eventplot([inference_item.loc_pred, x,y_hat], colors=["g", "b", "r"])
#xlim([0, 800])

In [ ]:
z = aligner.alignment_ref
y = (z-z[0]) / (scaler*335)
y_rev = np.sort(image_input.shape[-1]-y)-offset

x = aligner.alignment_qry/335
x_rev = np.sort(image_input.shape[-1]-x)

func_rev = interp1d(x=y_rev,y=x_rev, bounds_error=False, fill_value='extrapolate')

ref = compare.refs[ref_id]
start, end = aligner.alignment_ref[[0, -1]]
start_index = np.argmax(ref >= start)
end_index = np.argmin(ref <= end)
ref_crop = ref[start_index:end_index]

z_hat = (ref_crop-ref_crop[0]) / (scaler*335)
z_hat_rev = np.sort(image_input.shape[-1]-z_hat)-offset

print(z_hat_rev)

y_hat = func_rev(z_hat_rev)

In [ ]:
figure(figsize=(30, 3))
image_input = selector.selected[Molecule].bionano_image.segment_image[0]
target_width = localizer_module.image_channels
source_width = image_input.shape[0] // 2 + 1
image_input = image_input[source_width - target_width // 2: source_width + target_width // 2 + 1]
imshow(image_input, aspect="auto", cmap="gray")
eventplot([inference_item.loc_pred, x_rev, y_hat], colors=["g", "b", "r"])
xlim([0,800])

In [ ]:
%load_ext autoreload
%autoreload 2
from deepom.data_fetcher import DataFetcher
num_mols = 20
data_fetcher = DataFetcher(num_molecules=num_mols)
# data_fetcher.generate_ground_truth_plots_simulated()
data_fetcher.generate_ground_truth_plots()
# data_fetcher.generate_ground_truth_simulated(2)

In [ ]:
# import image module
from IPython.display import Image,display
  
# get the image
for img_index in range(num_mols):
    display(Image(url=f"{data_fetcher.save_path}/ground_truth_{img_index}.png"))
    
    